# `ETraceOp`: 在线学习算子

在`braintrace`框架中，资格迹算子 (`ETraceOp`) 扮演着连接神经网络中神经元群体、定义突触交互的核心角色。它的主要职责是根据模型的输入（突触前活动）和参数（如突触权重），精确计算出突触后电流。更重要的是，`ETraceOp` 原生支持基于资格迹（Eligibility Trace）的学习机制，这是一种模拟生物神经系统中时间信用分配（temporal credit assignment）的关键过程，使得模型能够根据延迟的奖励或误差信号来更新连接权重。

`ETraceOp` 的设计哲学是将计算逻辑（算子本身）与可训练参数（`ETraceParam`）解耦，从而提供了极大的灵活性和可扩展性。

In [35]:
import brainevent
import brainstate
import jax
import jax.numpy as jnp

import braintrace

## 内置的资格迹算子

`braintrace` 提供了一系列功能强大且预先配置好的资格迹算子，能够满足绝大多数常见的神经网络建模需求。这些算子与模型参数容器 `braintrace.ETraceParam` 配合使用，构成了神经网络的构建模块。

主要内置算子包括：


- [`braintrace.MatMulOp`](../apis/generated/braintrace.MatMulOp.rst): 实现标准的矩阵乘法，是构建全连接层（Dense Layer）的基础。
- [`braintrace.ConvOp`](../apis/generated/braintrace.ConvOp.rst): 实现卷积操作，支持1D、2D和3D卷积，是构建卷积神经网络（CNN）的核心。
- [`braintrace.SpMatMulOp`](../apis/generated/braintrace.SpMatMulOp.rst): 专为稀疏连接设计，实现了稀疏矩阵乘法，在图神经网络（GNN）和需要高效表示大规模稀疏连接的生物可塑性模型中尤为重要。
- [`braintrace.ElemWiseOp`](../apis/generated/braintrace.ElemWiseOp.rst): 执行元素级别的数学运算，常用于实现激活函数、缩放或其他自定义的逐元素变换。
- [`braintrace.LoraOp`](../apis/generated/braintrace.LoraOp.rst): 实现低秩适应（Low-Rank Adaptation）技术，这是一种高效微调大型预训练模型的方法。

这些资格迹算子通常需要配合模型参数`braintrace.ETraceParam`一起使用。

### `braintrace.MatMulOp` 矩阵乘法算子

[`braintrace.MatMulOp`](../apis/generated/braintrace.MatMulOp.rst) 是最基础的算子，支持矩阵乘法操作，适用于全连接层等场景。

**基本操作**：
- 输入：矩阵 $x \in \mathbb{R}^{B \times D_{in}}$
- 参数：字典 $w$，包含权重矩阵 `weight` $\in \mathbb{R}^{D_{in} \times D_{out}}$ 和偏置向量 `bias` $\in \mathbb{R}^{D_{out}}$
- 输出：矩阵 $y \in \mathbb{R}^{B \times D_{out}}$

**支持的操作类型**：

1. **标准矩阵乘法**：

$$y = x \cdot \text{param['weight']} + \text{param['bias']}$$


In [36]:
# 标准矩阵乘法

braintrace.ETraceParam(
    {
        'weight': brainstate.random.rand(4, 5),
        'bias': brainstate.random.rand(5)
    },
    braintrace.MatMulOp()
)

ETraceParam(
  value={
    'bias': ShapedArray(float32[5]),
    'weight': ShapedArray(float32[4,5])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=MatMulOp(
    is_diagonal=False,
    weight_mask=None,
    weight_fn=<function MatMulOp.<lambda> at 0x000002380AAB7B00>,
    apply_weight_fn_before_mask=False
  ),
  is_etrace=True
)

2. **掩码操作**： 通过 `weight_mask` 参数可以实现稀疏连接，只有掩码中为`True`的权重才会生效。

$$y = x \cdot (\text{param['weight']} \odot \text{mask}) + \text{param['bias']}$$

In [37]:
# 带掩码的矩阵乘法（实现稀疏连接）

braintrace.ETraceParam(
    {
        'weight': brainstate.random.rand(4, 5),
        'bias': brainstate.random.rand(5)
    },
    braintrace.MatMulOp(
        weight_mask=brainstate.random.rand(4, 5) > 0.5
    )
)

ETraceParam(
  value={
    'bias': ShapedArray(float32[5]),
    'weight': ShapedArray(float32[4,5])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=MatMulOp(
    is_diagonal=False,
    weight_mask=Array([[ True, False,  True,  True,  True],
           [False, False,  True,  True,  True],
           [ True, False, False, False, False],
           [False,  True, False, False,  True]], dtype=bool),
    weight_fn=<function MatMulOp.<lambda> at 0x000002380AAB7B00>,
    apply_weight_fn_before_mask=False
  ),
  is_etrace=True
)

3. **权重函数变换**：通过 `weight_fn` 参数可以对权重矩阵应用一个函数变换，例如应用`jnp.abs`来强制执行戴尔定律（Dale's Law），确保所有突触权重为正（兴奋性）。

$$y = x \cdot f(\text{param['weight']}) + \text{param['bias']}$$


In [38]:
# 对权重应用函数变换

braintrace.ETraceParam(
    {
        'weight': brainstate.random.rand(4, 5),
        'bias': brainstate.random.rand(5)
    },
    braintrace.MatMulOp(
        weight_fn=jnp.abs   # 确保权重为正
    )
)

ETraceParam(
  value={
    'bias': ShapedArray(float32[5]),
    'weight': ShapedArray(float32[4,5])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=MatMulOp(
    is_diagonal=False,
    weight_mask=None,
    weight_fn=<PjitFunction of <function abs at 0x000002386178C400>>,
    apply_weight_fn_before_mask=False
  ),
  is_etrace=True
)

4. **组合操作**：

$$y = x \cdot f(\text{param['weight']} \odot \text{mask}) + \text{param['bias']}$$

In [39]:
# 同时使用掩码和权重函数
braintrace.ETraceParam(
    {
        'weight': brainstate.random.rand(4, 5),
        'bias': brainstate.random.rand(5)
    },
    braintrace.MatMulOp(
        weight_fn=jnp.abs,
        weight_mask=brainstate.random.rand(4, 5) > 0.5
    )
)

ETraceParam(
  value={
    'bias': ShapedArray(float32[5]),
    'weight': ShapedArray(float32[4,5])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=MatMulOp(
    is_diagonal=False,
    weight_mask=Array([[False, False,  True,  True, False],
           [ True,  True,  True, False,  True],
           [ True, False, False, False, False],
           [ True,  True, False, False, False]], dtype=bool),
    weight_fn=<PjitFunction of <function abs at 0x000002386178C400>>,
    apply_weight_fn_before_mask=False
  ),
  is_etrace=True
)

### ``braintrace.ConvOp`` 卷积算子

[`braintrace.ConvOp`](../apis/generated/braintrace.ConvOp.rst) 算子支持一般性的卷积操作，适用于卷积神经网络（CNN）等场景。它的输入是特征图$x$和参数$w$，输出是卷积结果$y$。

- 输入$x$是一个矩阵。
- 参数$w$是一个字典，涵盖了权重矩阵字段``weight``和偏置向量字段``bias``。这个算子可以用于实现全连接层的前向传播。
- 输出$y$是一个矩阵。

**维度支持**：

`braintrace.ConvOp`支持1D、2D、3D卷积等多种形式的卷积操作。通过 `xinfo` 参数（一个`jax.ShapeDtypeStruct`对象），它可以自动推断并执行1D、2D或3D卷积。比如，

- **1D卷积**：当 `xinfo=jax.ShapeDtypeStruct((32, 3), jnp.float32)` 时，表示输入是一个形状为 `(32, 3)` 的2维张量（通道数为3，长度均为32），此时卷积是1D卷积。
- **2D卷积**：当 `xinfo=jax.ShapeDtypeStruct((32, 32, 3), jnp.float32)` 时，表示输入是一个形状为 `(32, 32, 3)` 的3维张量（通道数为3，高度和宽度均为32），此时卷积是2D卷积。
- **3D卷积**：当 `xinfo=jax.ShapeDtypeStruct((32, 32, 32, 3), jnp.float32)` 时，表示输入是一个形状为 `(32, 32, 32, 3)` 的4维张量（通道数为3，高度和宽度均为32，深度为32），此时卷积是3D卷积。



**1. 标准卷积操作**：

$$y = x \star \text{param['weight']} + \text{param['bias']}$$

其中 $\star$ 表示卷积操作。

In [40]:
# 以2D卷积为例
braintrace.ETraceParam(
    {
        'weight': brainstate.random.rand(3, 3),
        'bias': jnp.zeros(16)
    },
    braintrace.ConvOp(
        xinfo=jax.ShapeDtypeStruct((32, 3, 3), jnp.float32),  # (height, width, channels)
        window_strides=[1, 1],
        padding='SAME',
    )
)

ETraceParam(
  value={
    'bias': ShapedArray(float32[16]),
    'weight': ShapedArray(float32[3,3])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=ConvOp(
    is_diagonal=False,
    window_strides=[
      1,
      1
    ],
    padding=SAME,
    lhs_dilation=None,
    rhs_dilation=None,
    feature_group_count=1,
    batch_group_count=1,
    dimension_numbers=None,
    weight_mask=None,
    weight_fn=<function ConvOp.<lambda> at 0x000002380AAB7EC0>,
    xinfo=ShapeDtypeStruct(shape=(32, 3, 3), dtype=float32)
  ),
  is_etrace=True
)

**2. `mask`操作**：

$$y = x \star  (\mathrm{param['weight']} * \mathrm{mask}) + \mathrm{param['bias']}$$


In [41]:
# 以2D卷积为例
braintrace.ETraceParam(
    {
        'weight': brainstate.random.rand(3, 3),
        'bias': jnp.zeros(16)
    },
    braintrace.ConvOp(
        xinfo=jax.ShapeDtypeStruct((32, 3, 3), jnp.float32),
        window_strides=[1, 1],
        padding='SAME',
        weight_mask=brainstate.random.rand(3, 3) > 0.5
    )
)

ETraceParam(
  value={
    'bias': ShapedArray(float32[16]),
    'weight': ShapedArray(float32[3,3])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=ConvOp(
    is_diagonal=False,
    window_strides=[
      1,
      1
    ],
    padding=SAME,
    lhs_dilation=None,
    rhs_dilation=None,
    feature_group_count=1,
    batch_group_count=1,
    dimension_numbers=None,
    weight_mask=Array([[False,  True, False],
           [False, False, False],
           [False,  True, False]], dtype=bool),
    weight_fn=<function ConvOp.<lambda> at 0x000002380AAB7EC0>,
    xinfo=ShapeDtypeStruct(shape=(32, 3, 3), dtype=float32)
  ),
  is_etrace=True
)

**3. 权重函数**：

$$y = x \star  f(\mathrm{param['weight']}) + \mathrm{param['bias']}$$

In [42]:
# 以2D卷积为例
braintrace.ETraceParam(
    {
        'weight': brainstate.random.rand(3, 3),
        'bias': jnp.zeros(16)
    },
    braintrace.ConvOp(
        xinfo=jax.ShapeDtypeStruct((32, 3, 3), jnp.float32),
        window_strides=[1, 1],
        padding='SAME',
        weight_fn=jnp.abs
    )
)

ETraceParam(
  value={
    'bias': ShapedArray(float32[16]),
    'weight': ShapedArray(float32[3,3])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=ConvOp(
    is_diagonal=False,
    window_strides=[
      1,
      1
    ],
    padding=SAME,
    lhs_dilation=None,
    rhs_dilation=None,
    feature_group_count=1,
    batch_group_count=1,
    dimension_numbers=None,
    weight_mask=None,
    weight_fn=<PjitFunction of <function abs at 0x000002386178C400>>,
    xinfo=ShapeDtypeStruct(shape=(32, 3, 3), dtype=float32)
  ),
  is_etrace=True
)

**4. 权重函数+mask操作**：

$$y = x \star  f(\mathrm{param['weight']} * \mathrm{mask}) + \mathrm{param['bias']}$$

In [43]:
# 以2D卷积为例
braintrace.ETraceParam(
    {
        'weight': brainstate.random.rand(3, 3),
        'bias': jnp.zeros(16)
    },
    braintrace.ConvOp(
        xinfo=jax.ShapeDtypeStruct((32, 3, 3), jnp.float32),
        window_strides=[1, 1],
        padding='SAME',
        weight_mask=brainstate.random.rand(3, 3) > 0.5,
        weight_fn=jnp.abs,
    )
)

ETraceParam(
  value={
    'bias': ShapedArray(float32[16]),
    'weight': ShapedArray(float32[3,3])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=ConvOp(
    is_diagonal=False,
    window_strides=[
      1,
      1
    ],
    padding=SAME,
    lhs_dilation=None,
    rhs_dilation=None,
    feature_group_count=1,
    batch_group_count=1,
    dimension_numbers=None,
    weight_mask=Array([[ True, False,  True],
           [False, False,  True],
           [False, False,  True]], dtype=bool),
    weight_fn=<PjitFunction of <function abs at 0x000002386178C400>>,
    xinfo=ShapeDtypeStruct(shape=(32, 3, 3), dtype=float32)
  ),
  is_etrace=True
)

### ``braintrace.SpMatMulOp`` 稀疏矩阵乘法算子

[`braintrace.SpMatMulOp`](../apis/generated/braintrace.SpMatMulOp.rst) 算子支持稀疏矩阵乘法操作，适用于图神经网络（GNN）等场景。它的输入是特征图$x$和参数$w$，输出是稀疏矩阵乘法结果$y$。

`braintrace.SpMatMulOp` 与 `braintrace.MatMulOp` 做类似的操作，用于进行矩阵乘法操作：

$$
y = x @ \mathrm{param['weight']} + \mathrm{param['bias']}
$$

只不过此时的`param['weight']` 是一个稀疏矩阵，通常是[``brainevent``](https://brainevent.readthedocs.io/)中实现的稀疏矩阵，在保持计算效率的同时，大幅降低内存消耗。包括：

- ``brainevent.CSR``: 压缩稀疏行矩阵（Compressed Sparse Row）。
- ``brainevent.CSC``: 压缩稀疏列矩阵（Compressed Sparse Column）。
- ``brainevent.COO``: 坐标格式稀疏矩阵（Coordinate Format）。


`braintrace.SpMatMulOp`支持如下操作：

**1. 标准矩阵乘法：**

$$y = x @ \mathrm{param['weight']} + \mathrm{param['bias']}$$

In [44]:
data = jnp.where(
    brainstate.random.rand(100, 100) < 0.2,
    brainstate.random.rand(100, 100),
    0.
)
csr = brainevent.CSR.fromdense(data)

In [45]:
braintrace.ETraceParam(
    {'weight': brainstate.random.rand(100)},
    braintrace.SpMatMulOp(csr)
)

ETraceParam(
  value={
    'weight': ShapedArray(float32[100])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=SpMatMulOp(
    is_diagonal=False,
    sparse_mat=CSR(float32[100, 100], nse=1981),
    weight_fn=<function SpMatMulOp.<lambda> at 0x000002380AAD42C0>
  ),
  is_etrace=True
)

**2. 权重函数：**

$$y = x @ f(\mathrm{param['weight']}) + \mathrm{param['bias']}$$

In [46]:
braintrace.ETraceParam(
    {'weight': brainstate.random.rand(100)},
    braintrace.SpMatMulOp(csr, weight_fn=jnp.abs)
)

ETraceParam(
  value={
    'weight': ShapedArray(float32[100])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=SpMatMulOp(
    is_diagonal=False,
    sparse_mat=CSR(float32[100, 100], nse=1981),
    weight_fn=<PjitFunction of <function abs at 0x000002386178C400>>
  ),
  is_etrace=True
)

### ``braintrace.ElemWiseOp`` 元素级操作算子

[`braintrace.ElemWiseOp`](../apis/generated/braintrace.ElemWiseOp.rst) 提供了一种简洁的方式来对参数进行逐元素的函数变换。它不直接处理来自突触前的输入 $x$，而是直接作用于其自身的参数 $w$。

**核心运算**：
$$y = f(w)$$
这可以用于创建可学习的激活函数参数、神经元阈值、时间常数等。

以下是一些典型的元素级操作算子示例：

In [47]:
braintrace.ETraceParam(
    brainstate.random.rand(4),
    braintrace.ElemWiseOp(jnp.abs)  # 绝对值操作
)

ETraceParam(
  value=ShapedArray(float32[4]),
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=ElemWiseOp(
    fn=<PjitFunction of <function abs at 0x000002386178C400>>,
    is_diagonal=True
  ),
  is_etrace=True
)

In [48]:
braintrace.ETraceParam(
    brainstate.random.rand(4),
    braintrace.ElemWiseOp(jnp.exp)  # 指数操作
)

ETraceParam(
  value=ShapedArray(float32[4]),
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=ElemWiseOp(
    fn=<PjitFunction of <function exp at 0x00000238617354E0>>,
    is_diagonal=True
  ),
  is_etrace=True
)

In [49]:
# 使用自定义的lambda函数

braintrace.ETraceParam(
    brainstate.random.rand(4),
    braintrace.ElemWiseOp(lambda x: x ** 2 + 1.)  # 自定义函数
)

ETraceParam(
  value=ShapedArray(float32[4]),
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=ElemWiseOp(
    fn=<function <lambda> at 0x0000023810D1F420>,
    is_diagonal=True
  ),
  is_etrace=True
)

## 自定义资格迹算子




尽管`braintrace`提供了一套完备的内置算子，但研究和应用中常常需要探索新颖的神经网络层或突触可塑性规则。为此，`braintrace`允许用户通过继承`braintrace.ETraceOp`基类来轻松创建自定义算子。

自定义一个算子，关键在于理解并实现两个核心方法：`xw_to_y`和`yw_to_w`。

1.  **`xw_to_y(self, x, w)`：定义前向传播**

      * **目的**：这个方法定义了算子的核心计算逻辑，即如何根据突触前输入 `x` 和算子参数 `w` 计算出突触后输出 `y`。它在功能上等同于标准深度学习框架中层的`forward`方法。
      * **数学表示**：$y = f(x, w)$。
      * **参数**：
          * `x`: 突触前神经元的活动（例如，脉冲、发放率或特征向量）。
          * `w`: 一个包含该算子所有可学习参数的字典（例如`{'weight': ..., 'bias': ...}`）。

![](../_static/etraceop-xw2y.png)

2.  **`yw_to_w(self, y, w)`：定义梯度/迹的传播**

      * **目的**：这个方法是资格迹学习机制的核心。它定义了来自突触后神经元的“学习信号”（通常是误差梯度）如何“流回”并影响到每一个参数的资格迹。它回答了这样一个问题：“输出 `y` 的变化对参数 `w` 的影响有多大？”
      * **数学表示**：$w\_{new} = g(y\_{grad}, w)$。此处的 $y\_{grad}$ 是一个抽象的“学习信号”，而 $w\_{new}$ 则代表了参数的更新方向或其资格迹。
      * **应用场景**: 该方法的计算结果将直接用于更新资格迹 $\boldsymbol{\epsilon}^{t}$，最终的权重更新 $\Delta w$ 将是学习信号（如奖励预测误差）与资格迹的乘积，即 $\Delta w \propto \text{LearningSignal} \cdot \boldsymbol{\epsilon}^{t}$。
      * **参数**：
          * `y`: 一个代表学习信号或梯度的向量，其维度与`xw_to_y`的输出维度相同。
          * `w`: 当前的参数字典。

![](../_static/etraceop-yw2w.png)


### 示例：从零开始构建 `CustomizedMatMul`

让我们通过一个具体的例子来演示如何自定义一个功能与`MatMulOp`相同的算子。


In [50]:
class CustomizedMatMul(braintrace.ETraceOp):
    """
    一个自定义的矩阵乘法资格迹算子。
    它实现了 y = x @ w['weight'] + w['bias'] 的计算。
    """
    def xw_to_y(self, x, w: dict):
        """
        前向传播：计算 y = x @ weight + bias
        """
        return jnp.dot(x, w['weight']) + w['bias']

    def yw_to_w(self, y, w: dict):
        """
        参数更新（用于资格迹）：计算学习信号如何影响权重和偏置。
        这类似于计算梯度 dL/dw = dL/dy * dy/dw。
        """

        # 对于权重 w['weight']，其梯度是输入 x 和输出梯度 y 的外积。
        # 这里，我们将 y 的维度从 (B, D_out) 扩展到 (B, 1, D_out)，
        # 将 x 的维度从 (B, D_in) 扩展到 (B, D_in, 1)，
        # 从而通过广播乘法 (x[..., None] * y[:, None, :]) 实现外积的批处理计算。
        # 此处为了简化，我们以一种更抽象的方式来表示这种依赖关系。
        y_expanded = jnp.expand_dims(y, axis=-2) # 形状变为 (..., 1, D_out)
        return {
            'weight': y_expanded * w['weight'], # 示例性的更新规则
            'bias': y * w['bias'] # 示例性的更新规则
        }

**代码解析**:

  * 在 `xw_to_y` 中，我们实现了标准的矩阵乘法加偏置的逻辑，这非常直观。
  * 在 `yw_to_w` 中，我们定义了更新规则。返回的字典的键和值的形状必须与原始参数`w`完全匹配。`jnp.expand_dims`在这里用于调整`y`的维度，以确保广播机制能正确地将`y`的影响应用到`w['weight']`的每个元素上。


### 使用自定义算子

定义好之后，`CustomizedMatMul`可以像任何内置算子一样，与`ETraceParam`结合使用，并无缝集成到`braintrace`的计算图中。

In [51]:
# 1. 实例化自定义算子
my_op = CustomizedMatMul()

# 2. 使用ETraceParam将算子与具体参数关联
param = braintrace.ETraceParam(
    {
        'weight': brainstate.random.rand(4, 5),  # D_in=4, D_out=5
        'bias': brainstate.random.rand(5)
    },
    my_op # 将自定义算子实例传入
)

# 3. 在模型中使用 (模拟)
# 创建一些模拟的输入数据
dummy_input = brainstate.random.rand(1, 4) # Batch=1, D_in=4

# braintrace的运行器会自动调用 op.xw_to_y(dummy_input, param.value)
# 我们可以手动调用来验证
output = my_op.xw_to_y(dummy_input, param.value)

print("自定义算子实例化:")
print(param)
print("\n前向计算输出:")
print(output)
print("输出形状:", output.shape)

自定义算子实例化:
ETraceParam(
  value={
    'bias': ShapedArray(float32[5]),
    'weight': ShapedArray(float32[4,5])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=CustomizedMatMul(
    is_diagonal=False
  ),
  is_etrace=True
)

前向计算输出:
[[1.3891417 1.7291917 2.2308984 0.7964938 1.805964 ]]
输出形状: (1, 5)


通过这种方式，您可以为新颖的神经元模型、复杂的突触可塑性规则或任何需要可微分参数和自定义计算逻辑的场景，构建高度定制化的组件。